# Import data

**Run this cell to transform raw file from bb to file without the non-utf8 characters �**

Removes � characters from file

In [650]:
# Open the file in read mode, read its contents, then close it
with open('../data/disaster-tweets.csv', 'r', encoding='utf-8', errors='ignore') as file:
    content = file.read()

# Remove all � characters
content = content.replace('�', '')

# Open the file in write mode and write the modified content back to it
with open('../data/disaster-tweets.csv', 'w', encoding='utf-8') as file:
    file.write(content)

In [651]:
import pandas as pd

data_path = '../data/disaster-tweets.csv'
# read csv, ","-separated

df = pd.read_csv(data_path, sep=',')

# print dimensions
print("Shape of raw data", df.shape)

# print first 2 rows
df.head(2)

Shape of raw data (10876, 13)


,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,choose_one,choose_one:confidence,choose_one_gold,keyword,location,text,tweetid,userid
0,778243823,True,golden,156,NaN,Relevant,1.0,Relevant,NaN,NaN,Just happened a terrible car crash,1.0,NaN
1,778243824,True,golden,152,NaN,Relevant,1.0,Relevant,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,13.0,NaN


In [652]:
# save text, id to csv
df[['text', 'tweetid']].to_csv('results/texts.csv', index=False, sep=';')

# save all relevant rows to csv, i.e. choose_one = Relevant
df[df['choose_one'] == 'Relevant'][['text', 'tweetid', 'location']].to_csv('results/relevant.csv', index=False, sep=';')

# Explore data

In [653]:
df.describe()

# missing values
df.isnull().sum()

_unit_id                     0
_golden                      0
_unit_state                  0
_trusted_judgments           0
_last_judgment_at           84
choose_one                   0
choose_one:confidence        0
choose_one_gold          10789
keyword                     87
location                  3638
text                         0
tweetid                      0
userid                      87
dtype: int64

### Result

# Preprocess data

### Select features

In [654]:
raw_features = ['_unit_id', '_golden', '_unit_state', '_trusted_judgments',
       '_last_judgment_at', 'choose_one', 'choose_one:confidence',
       'choose_one_gold', 'keyword', 'location', 'text', 'tweetid', 'userid']

features_to_keep = [ 'choose_one', 'keyword', 'location', 'text' ]

df = df[features_to_keep]

### Choose_one

In [655]:
# drop rows with choose_one = 'Can't Decide'
df = df[df['choose_one'] != "Can't Decide"]

# if choose one is relevant, set to 1, else 0
df['choose_one'] = df['choose_one'].apply(lambda x: 1 if x == 'Relevant' else 0)

df['y'] = df['choose_one']
df.drop(columns=['choose_one'], inplace=True, errors='ignore')

df.head()

,keyword,location,text,y
0,NaN,NaN,Just happened a terrible car crash,1
1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
2,NaN,NaN,"Heard about #earthquake is different cities, s...",1
3,NaN,NaN,"there is a forest fire at spot pond, geese are...",1
4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1


### Text processing


In [656]:
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
import string
import re

tokenizer = TweetTokenizer(
    preserve_case=False,
    strip_handles=True,
    reduce_len=True,
)

Stopwords = set(stopwords.words('english'))

def remove_noise(doc):
    filter_words = []

    doc = doc.replace('�', '')

    # # Remove URLs
    doc = re.sub(r'http\S+', '', doc)
    doc = re.sub(r'www\.\S+', '', doc)

    tokenized = tokenizer.tokenize(doc)

    for word in tokenized:
        if (word not in Stopwords 
            and word not in string.punctuation 
            and not word.startswith('#')
            ):

            filter_words.append(word)

    return " ".join(filter_words)
    

df['text'] = df['text'].apply(remove_noise)

print("Dimensions after preprocessing", df.shape)
df.head()

Dimensions after preprocessing (10860, 4)


,keyword,location,text,y
0,NaN,NaN,happened terrible car crash,1
1,NaN,NaN,deeds reason may allah forgive us,1
2,NaN,NaN,heard different cities stay safe everyone,1
3,NaN,NaN,forest fire spot pond geese fleeing across str...,1
4,NaN,NaN,forest fire near la ronge sask canada,1


In [657]:
# save to csv where y=1
df[df['y'] == 1][['text', 'location']].to_csv('results/preprocessed_relevant.csv', index=False, sep=';')


## Bag of words

In [658]:
# Implement bag of words

import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()


bow = vectorizer.fit_transform(df['text'])

print("Shape of X", bow.shape)

# split into train and test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(bow, df['y'], test_size=0.2, random_state=42)

print("Shape of X_train", X_train.shape)
print("Shape of X_test", X_test.shape)
print("Shape of y_train", y_train.shape)
print("Shape of y_test", y_test.shape)

Shape of X (10860, 16340)
Shape of X_train (8688, 16340)
Shape of X_test (2172, 16340)
Shape of y_train (8688,)
Shape of y_test (2172,)


# Logistic regression model

In [659]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=42, solver='lbfgs').fit(X_train, y_train)

y_pred = clf.predict_proba(X_test)

# get probabilities for class 1
y_pred = y_pred[:, 1]

print(y_pred.tolist())

[0.1065275262012065, 0.7696939852908815, 0.04167206575491168, 0.21246169129542608, 0.1722190036779433, 0.09168005458256699, 0.41247631030662146, 0.9951656869923802, 0.12449956016867794, 0.16285228306936658, 0.13682910665076212, 0.10038497383241635, 0.0916547493485211, 0.8641471949606693, 0.3254573953618824, 0.9654796003294809, 0.9318596123612222, 0.986681787307879, 0.4755004972040417, 0.3546809052585615, 0.25551767370341383, 0.11332946357650117, 0.09208568239039835, 0.2615821107613064, 0.01698701387570849, 0.28073195283856417, 0.5963318148086729, 0.9468893831460833, 0.7136686863325, 0.43894035794587166, 0.1618239397217533, 0.17517689711112502, 0.4297258260867183, 0.08212666484218711, 0.12343371603372134, 0.8831835688863349, 0.1404126582747495, 0.974917002348988, 0.18502328341956617, 0.06565454483952636, 0.09555587738158809, 0.9709767780802158, 0.9974616617595901, 0.9993932836245489, 0.19375759119004785, 0.9185577810867714, 0.17874370886228552, 0.09133046075328874, 0.10568185287526369, 

### Performance metrics

In [660]:
from sklearn.metrics import accuracy_score, precision_score

clipped_y_pred = np.array([1 if x >= 0.5 else 0 for x in y_pred])

accuracy = accuracy_score(y_test, clipped_y_pred)
print("Accuracy on the test set:", accuracy)

precision = precision_score(y_test, clipped_y_pred, pos_label=1)

print("Precision on the test set:", precision)


Accuracy on the test set: 0.7808471454880295
Precision on the test set: 0.7931034482758621


### Exploring results

In [661]:
# map weights to vocabulary
weights = clf.coef_[0]
feature_names = vectorizer.get_feature_names_out()

feature_weights = pd.DataFrame({'feature': feature_names, 'weight': weights})

feature_weights.sort_values(by='weight', ascending=False, inplace=True)

feature_weights.head(20)

,feature,weight
6979,hiroshima,2.358691
5738,fires,2.313306
15919,wildfire,2.163371
15108,typhoon,2.098453
2217,bombing,2.039975
4236,derailment,2.003807
11720,rainstorm,1.865599
4847,earthquake,1.826262
13891,storm,1.808118
5255,evacuated,1.792753


In [662]:
import numpy as np

df_test = df.iloc[int(0.8*m):]

df_test['y_pred'] = y_pred

incorrectly_classified_indices = np.where(y_pred != y_test)[0]

# select rows df incorrectly_classified_indices
df_test.iloc[incorrectly_classified_indices].to_csv('results/incorrectly_classified_rows.csv', index=False)

# false negatives
false_negatives = np.where((y_pred == 0) & (y_test == 1))[0]

df_test.iloc[false_negatives].to_csv('results/false_negatives.csv', index=False)

# false positives

false_positives = np.where((y_pred == 1) & (y_test == 0))[0]

df_test.iloc[false_positives].to_csv('results/false_positives.csv', index=False)

/var/folders/gz/34s3tc0d1qz3drk0v83_097h0000gn/T/ipykernel_71825/1980399844.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['y_pred'] = y_pred
